In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os.path import join, basename
from scipy.io import savemat
from skimage import img_as_ubyte
from skimage.transform import resize
from skimage.segmentation import expand_labels
from skimage.measure import regionprops_table, perimeter, perimeter_crofton
from skimage.morphology import remove_small_holes, remove_small_objects
from skimage.filters import threshold_otsu, threshold_multiotsu, unsharp_mask

from cccode.image import ck
from Deeplearning.evaluate import RecognitionAnalyzer
from Deeplearning.util.functions import source_from_sample_id, figure_preproduction
from Deeplearning.bloodsmear import (outlier_cells_visual, select_outlier_cells, 
    remove_cells_closed_to_edge, cell_region_properties)

WAVELENGTH = 532e-9
ProjectRoot = "D:\\Workspace\\Blood Recognition\\Deeplearning"
FiguresRoot = "D:\\Postgraduate\\Investigate\\Blood Recognition\\figures"

### Outlier Cells Extracting for Individual Samples

In [2]:
# # The inputs of func 'cell_region_properties' is composed of area labels
def outlier_cell_area_mask(image, boxes, indexes=None, otsu_bins=100, expand_distance=6):
    # Pick out the boxes of outlier cells
    if indexes is not None:
        boxes = boxes[indexes]

    # Extracting the available area from the given boxes
    labeled_mask = np.zeros_like(image)
    for lbl, (x, y, w, h, _) in enumerate(boxes):
        # notice: the 'lbl' used here is different from the 'label' used in object annotating
        # extract available areas from boxes
        y_min, y_max = int(y-h/2), int(y+h/2)
        x_min, x_max = int(x-w/2), int(x+w/2)
        limited_area = image[y_min:y_max, x_min:x_max]
        otsu_threshold = threshold_otsu(limited_area, nbins=otsu_bins)
        # labeling available areas
        y_index, x_index = (limited_area >= otsu_threshold).nonzero()
        labeled_mask[y_index+y_min, x_index+x_min] = lbl + 1
        
    # expanding the extracted areas
    expanded_mask = expand_labels(labeled_mask, distance=expand_distance).astype(int)

    # Before image area extracting, normalize the image to (0, 1), and amend image
    image = (image-np.min(image)) / (np.max(image)-np.min(image))  # Normalize
    image = unsharp_mask(image, radius=5, amount=1)                # Unsharp
    image = image - np.min(image)
    
    # Exact cell area extracting
    for lbl in range(1, np.max(expanded_mask)+1):
        roi = image * (expanded_mask == lbl)
        sep_threshold = threshold_otsu(roi)
        regions = np.digitize(roi, bins=(np.min(roi), sep_threshold, np.max(roi)))
        removed_area = (regions == 1) * (expanded_mask == lbl)
        expanded_mask[removed_area] = 0
        
        # Smoth cell center holes and remove surrounding dots
        cur_lbl = expanded_mask == lbl
        cur_lbl_restored = remove_small_holes(cur_lbl, 5000)
        cur_lbl_restored = remove_small_objects(cur_lbl_restored, 3000).astype(int)
        res = cur_lbl - cur_lbl_restored
        expanded_mask -= res * lbl
    return expanded_mask

### Load Resource

In [3]:
analyzed_data_fname = 'caches\\acc_compare_set202109_1211-141810.pkl'
analyzer = RecognitionAnalyzer(join(ProjectRoot, analyzed_data_fname))

source_fnames = analyzer.data.get("source_fnames")
bsrnet_recognitions = analyzer.data.get("BSRNet")

### Cell Properties Analysis

In [4]:
def sample_rescale(image, boxes, scale=10):
    image = resize(image, [ax*scale for ax in image.shape])
    for i, box in enumerate(boxes):
        box = [elm*scale for elm in box]
        boxes[i] = box
    return image, boxes


selected_fnames, phases, labels = [], [], []
for i, (fname, boxes) in enumerate(zip(source_fnames, bsrnet_recognitions)):
    print(f"\r", i, end="")
    phase = source_from_sample_id(basename(fname), "phase_matrix")
    # Select outlier cells and only focus on rbc
    outlier_rbc_indexes = select_outlier_cells(boxes, specified_type_idx=0)
    outlier_rbc_indexes = remove_cells_closed_to_edge(boxes, outlier_rbc_indexes, phase.shape)
    # Extract rbc cell region properties based on cell coordinates and phase
    if len(outlier_rbc_indexes) > 0:
        selected_fnames.append(fname)
        # outlier_cells_visual(phase, boxes[outlier_rbc_indexes])
        # phase, boxes = sample_rescale(phase, boxes)
        cells_area_mask = outlier_cell_area_mask(phase, boxes, outlier_rbc_indexes)
        # ck.img_show(phase, cells_area_mask)
        phases.append(phase)
        labels.append(cells_area_mask)

 122

#### Data transfer to Matlab

In [197]:
# phase normalize
phase = (phase-phase.min())/(phase.max()-phase.min())
phase = img_as_ubyte(phase)

import warnings
warnings.filterwarnings("ignore")
cells_area_mask = img_as_ubyte(cells_area_mask)

savemat(join(FiguresRoot, "statistics\\pre10_pha-lbl.mat"),
        {"phase_pre10": phases[:10],
         "label_pre10": labels[:10]})

#### Cell region properties

In [5]:
PixelSize = 4.8e-6

def cell_region_properties(labeled_areas, phase, properties=None) -> dict:
    """ Extract recognized RBC cells with the dominant areas and
    produce specified properties into and pandas dict for every phase image.
    Returns
    -------
    properties_tables: list
        [properties_table_0: {'area', 'bbox', ...}, properties_table_1, ...] """
    def volume(image, intensity): return np.sum(intensity * image * PixelSize**2)

    def mch(image): return 10 * WAVELENGTH * np.sum(image > 0) / (2 * np.pi * 0.002)

    def mean_phase_shift(image, intensity): return np.sum(intensity) / np.sum(image)

    def form_factor(image): return 4 * np.pi * np.sum(image) / np.square(perimeter(image, 4))

    if not properties:
        properties = ("image", "label", "area", "bbox", "eccentricity", "perimeter", "intensity_image")
    prop_dict = regionprops_table(labeled_areas, 
                                  intensity_image=phase, 
                                  properties=properties,
                                  extra_properties=(form_factor, volume, mean_phase_shift, mch))
    return prop_dict

In [6]:
rbc_properites = {}
for phase, label in zip(phases, labels):
    # Region properties
    prop_dict = cell_region_properties(label, phase)
    # prop_dict["cell_identity"] = [(fname, idx) for idx in outlier_rbc_indexes]
    for k, v in prop_dict.items():
        if k not in rbc_properites.keys():
            rbc_properites[k] = v
        else:
            rbc_properites[k] = np.concatenate([rbc_properites[k], v])

C:\Users\chenchao\AppData\Local\Temp/ipykernel_5712/1700443598.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  def form_factor(image): return 4 * np.pi * np.sum(image) / np.square(perimeter(image, 4))


In [9]:
with open("./Resourse/UpsampledRBCProps.pkl", "wb") as f:
    pickle.dump(rbc_properites, f)

In [8]:
def cell_collections_visualization(images):
    fig, axes = plt.subplots(4, 8, figsize=(16, 8), constrained_layout=True)
    for i, img in enumerate(images):
        nr, nc = divmod(i, 8)
        ax = axes[nr, nc]
        ax.imshow(img, cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
        for sp in ["top", "bottom", "left", "right"]:
            ax.spines[sp].set_visible(False)
    plt.show()

intensitys = rbc_properites["image"]
for i in range(len(intensitys)//32):
    cell_collections_visualization(intensitys[i*32:(i+1)*32])